In [1]:
import sys
sys.path.append('../../python')
import thanos
from thanos import nn, init, ops
import torch
import numpy as np

In [2]:
#!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

In [3]:
# read it in to inspect it
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [4]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  1115394


In [5]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

In [6]:
data = thanos.Tensor(encode(text))
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [7]:
# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = thanos.stack([data[i:i+block_size] for i in ix])
    y = thanos.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

In [8]:
class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(0.5),
        )

    def forward(self, x):
        return self.net(x)

In [9]:
class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = nn.MultiheadAttention(n_embd, n_head)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        lx = self.ln1(x)
        lx, _ = self.sa(lx)
        x = x + lx
        x = x + self.ffwd(self.ln2(x))
        return x

In [10]:
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0
# ------------

In [90]:
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(thanos.Tensor(np.arange(T))) # (T,C)
        pos_emb = thanos.reshape(pos_emb, (1,) + pos_emb.shape)
        pos_emb = thanos.broadcast_to(pos_emb, tok_emb.shape)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = thanos.reshape(logits, (B*T, C))
            targets = thanos.reshape(targets, (B*T,))
            loss = nn.SoftmaxLoss()(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # NOT WORK
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            print(idx.shape, block_size)
            idx_cond = idx[:, -min(idx.shape[1], block_size):]
            # get the predictions
            logits, loss = self(idx_cond)
            logits = logits.detach()
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = nn.Softmax(dim=2)(logits.detach()) # (B, C)
            # sample from the distribution
            probs = thanos.reshape(probs, (1, vocab_size))
            idx_next = torch.multinomial(torch.tensor(probs.detach().numpy()), num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx_next = thanos.Tensor(idx_next.detach().numpy())
            print(idx, idx_next)
            idx = thanos.stack((idx, idx_next), axis=1) # (B, T+1)
            idx = thanos.reshape(idx, (idx.shape[0], idx.shape[1]))
        return idx
model = BigramLanguageModel()
optimizer = thanos.optim.Adam(model.parameters(), lr=learning_rate)

In [91]:
for iter in range(max_iters):
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    if iter % 100 == 0:
        print("step:", iter, " loss:", loss.detach().numpy())
        context = thanos.init.zeros(1, 1)
        print(decode(model.generate(context, max_new_tokens=200)[0].tolist()))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

step: 0  loss: 5.0024214
(1, 1) 32
[[0.]] [[47.]]
(1, 2) 32


TypeError: compact(): incompatible function arguments. The following argument types are supported:
    1. (arg0: thanos.backend_ndarray.ndarray_backend_cpu.Array, arg1: thanos.backend_ndarray.ndarray_backend_cpu.Array, arg2: List[int], arg3: List[int], arg4: int) -> None

Invoked with: <thanos.backend_ndarray.ndarray_backend_cpu.Array object at 0x134567d30>, <thanos.backend_ndarray.ndarray_backend_cpu.Array object at 0x134822f70>, (1, 1, 65), (130, 65, 1), -65

In [73]:
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))

NameError: name 'm' is not defined